In [24]:
%pip install -q --upgrade pip
%pip install -q piper-tts

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import shutil, sys
print("piper CLI:", shutil.which("piper"))
try:
    import piper
    print("piper module:", getattr(piper, "__version__", "import ok"))
except Exception as e:
    print("import error:", e)

piper CLI: /Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/venv/bin/piper
piper module: import ok


In [25]:

from faster_whisper import WhisperModel
model = WhisperModel("medium", device="cpu", compute_type="int8")  # great on Apple Silicon
segments, info = model.transcribe("WhisperDemoHi.m4a", task="translate", language="hi")  # or task="transcribe"
text = "".join(s.text for s in segments)
print(text)

 Hi, Namaste. My name is Kartik. What is your name? What else should I say or should I keep talking? I went to school the day before yesterday and I was studying a lot. When I was studying, I had an injury in my leg and I was not able to walk. So what should I do? Enough.


In [44]:
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np
import os

file_path = "/Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/input.wav"
if os.path.exists(file_path):
    os.remove(file_path)

SAMPLE_RATE = 16000
DURATION_S  = 5
OUT_WAV     = "input.wav"

print("🎙️ Recording... speak now")
audio = sd.rec(int(DURATION_S * SAMPLE_RATE), samplerate=SAMPLE_RATE, channels=1, dtype='float32')
sd.wait()
audio_i16 = (audio * 32767).astype("int16")
write(OUT_WAV, SAMPLE_RATE, audio_i16)
print(f"✅ Saved {OUT_WAV}")

from faster_whisper import WhisperModel
model = WhisperModel("medium", device="cpu", compute_type="int8")  # great on Apple Silicon
segments, info = model.transcribe("input.wav", task="translate", language="hi")  # or task="transcribe"
text = "".join(s.text for s in segments)
print(text)

🎙️ Recording... speak now
||PaMacCore (AUHAL)|| Warning on line 521: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Warning on line 441: err=''!obj'', msg=Unknown Error
||PaMacCore (AUHAL)|| Error on line 1322: err='-10851', msg=Audio Unit: Invalid Property Value


PortAudioError: Error opening InputStream: Internal PortAudio error [PaErrorCode -9986]

In [43]:
import shutil, sys, subprocess
from IPython.display import Audio, display
from pathlib import Path

PIPER = shutil.which("piper") or [sys.executable, "-m", "piper"]
VOICE = str(Path("voices/en_US-hfc_male.onnx").resolve())

cmd = ([PIPER] if isinstance(PIPER, str) else PIPER) + ["-m", VOICE, "-f", "hello.wav"]
subprocess.run(cmd, input="Hello from Piper.".encode("utf-8"), check=True)
display(Audio("hello.wav", autoplay=True))


Traceback (most recent call last):
  File "/Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/venv/bin/piper", line 7, in <module>
    sys.exit(main())
             ~~~~^^
  File "/Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/venv/lib/python3.13/site-packages/piper/__main__.py", line 136, in main
    raise ValueError(
        f"Unable to find voice: {model_path} (use piper.download_voices)"
    )
ValueError: Unable to find voice: /Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/voices/en_US-hfc_male.onnx (use piper.download_voices)


CalledProcessError: Command '['/Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/venv/bin/piper', '-m', '/Users/mukundsenthilkumar/Documents/Offline-AI-Kiosk/voices/en_US-hfc_male.onnx', '-f', 'hello.wav']' returned non-zero exit status 1.

In [42]:
import json, urllib.request

INDEX_URL = "https://huggingface.co/rhasspy/piper-voices/raw/main/voices.jsonl"

def load_index():
    req = urllib.request.Request(INDEX_URL, headers={"User-Agent": "Mozilla/5.0"})
    lines = urllib.request.urlopen(req).read().decode().splitlines()
    return [json.loads(l) for l in lines]

voices = load_index()

# Show some English voices (name | quality | speaker if provided)
for v in voices:
    if v["language"]["code"].startswith("en"):
        print(f'{v["model"]["name"]:<30} | {v["quality"]:<6} | {v.get("speaker","")}')

HTTPError: HTTP Error 404: Not Found